# Tidy Data in Python
by [Jean-Nicholas Hould](http://www.jeannicholashould.com/) with modifications by Ortal Dayan for Applied ML in Healthcare course 

In [18]:
import pandas as pd
import datetime
import os
from os import listdir
from os.path import isfile, join
import glob
import re

#path to data file
GETCWD = os.getcwd()
path_to_data = os.path.join(GETCWD + "\\Data")

# Force printing all the variables in each cell, not just the last one: 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### Summary – Requirements for Tidy Dataset

<br>

There are 3 main requirements for a tidy dataset:
- Each column stores a variable 
- Each row stores an observation
- Each cell stores a single value
<br>

Messy datasets are in violation of these 3 rules. 
<br>
<br>

The 5 most common problems with messy datasets:
- Column headers are values, not variable names
- Multiple variables are stored in one column
- Variables are stored in both rows and columns
- Multiple types of observational units are stored in the same table
- A single observational unit is stored in multiple tables

## 1. Variabels Should Be In Columns and Observations In Rows

In [19]:
df_imaginary = pd.read_csv(path_to_data + "\\imaginary-raw.csv", index_col = 0)
df_imaginary

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


### Transposing a table/dataframe 

In [22]:
df_imaginary.T

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


### Assigning indices (names) as a column and resetting the index

In [4]:
#print(df_imaginary)
df_imaginary_formatted = df_imaginary.copy()
df_imaginary_formatted.insert(0, 'name', df_imaginary.index)
df_imaginary_formatted.reset_index(inplace=True, drop=True)
df_imaginary_formatted

,name,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


### Melting df_imaginary

In [5]:
# df_imaginary_formatted = pd.melt(df_imaginary_formatted, id_vars=['name'], value_vars=['treatmenta', 'treatmentb'], 
#                        var_name = 'trt', value_name = 'result')
df_imaginary_formatted = pd.melt(df_imaginary_formatted, ['name'],  var_name = 'trt', value_name = 'result')
df_imaginary_formatted

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


## 2. Column Headers are Values, Not Variable Names

### Pew Research Center Dataset

In [6]:
df_pew = pd.read_csv(path_to_data + "pew-raw.csv")
df_pew

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


In [7]:
formatted_df_pew = pd.melt(df_pew, 'religion', var_name='income', value_name='freq')
formatted_df_pew = formatted_df_pew.sort_values(by=["religion"])
formatted_df_pew.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


### Billboard Top 100 Dataset

In [8]:
df_billboard = pd.read_csv(path_to_data + "billboard-raw.csv", encoding="mac_latin2")
df_billboard = df_billboard.rename(columns={'artist.inverted': 'artist'})
df_billboard.head(8)

,year,artist,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Melting
id_vars = ["year","artist","track","time",'genre',"date.entered","date.peaked"]
df_billboard = df_billboard.melt(id_vars=id_vars, var_name="week", value_name="rank")
#df_billboard.head(5)

# Cleaning out unnecessary rows by dropping rows with any NaNs
df_billboard = df_billboard.dropna()
#df_billboard.head(5)

# # Formatting 
df_billboard["week"] = df_billboard["week"].str.extract('(\d+)').astype(int) # astype(int, errors='ignore') 
df_billboard["week"] 

df_billboard["rank"] = df_billboard["rank"].astype(int, errors='ignore') # ignore : suppress exceptions. On error return original object.

# Create "date" columns
df_billboard['date'] = pd.to_datetime(df_billboard['date.entered']) + pd.to_timedelta(df_billboard['week'], unit='w') - pd.DateOffset(weeks=1)

df_billboard = df_billboard[["year", "artist", "time", "track", "date", "week", "rank"]]
df_billboard = df_billboard.sort_values(ascending=True, by=["year", "artist", "track", "week", "rank"])

# Assigning the tidy dataset to a variable for future usage
df_b_board = df_billboard

df_billboard.head(15)

0         1
1         1
2         1
3         1
4         1
         ..
19663    63
19700    63
19980    64
20017    64
20334    65
Name: week, Length: 5307, dtype: int64

,year,artist,time,track,date,week,rank
246,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-02-26,1,87
563,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-04,2,82
880,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-11,3,72
1197,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-18,4,77
1514,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-25,5,87
1831,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-01,6,94
2148,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-08,7,99
287,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-02,1,91
604,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-09,2,87
921,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-16,3,92


## 3. Multiple types in one table

In [10]:
songs_cols = ["artist", "track", "time"]
df_songs = df_b_board[songs_cols]
df_songs = df_b_board[songs_cols].drop_duplicates()
df_songs = df_songs.reset_index(drop=True)
df_songs.insert(0, "song_id", df_songs.index)
df_songs.head(15)

,song_id,artist,track,time
0,0,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22
1,1,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15
2,2,3 Doors Down,Kryptonite,3:53
3,3,3 Doors Down,Loser,4:24
4,4,504 Boyz,Wobble Wobble,3:35
5,5,98°,Give Me Just One Night (Una Noche),3:24
6,6,A*Teens,Dancing Queen,3:44
7,7,Aaliyah,I Don't Wanna,4:15
8,8,Aaliyah,Try Again,4:03
9,9,"Adams, Yolanda",Open My Heart,5:30


In [11]:
ranks = pd.merge(df_b_board, df_songs, on=["artist", "track", "time"])
ranks = ranks[["song_id","date","rank"]] # song id can be used as a key for all repetetive values which are the same per song_id: year, artist, time track 
ranks.head(15)

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87
5,0,2000-04-01,94
6,0,2000-04-08,99
7,1,2000-09-02,91
8,1,2000-09-09,87
9,1,2000-09-16,92


In [12]:
ranks = df_b_board.copy()
ranks['song_id'] = ranks.groupby(["artist", "track", "time"]).ngroup()
ranks
ranks = ranks[["song_id","date","rank"]]
ranks.reset_index(drop=True, inplace=True)
ranks.head(15)

,year,artist,time,track,date,week,rank,song_id
246,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-02-26,1,87,0
563,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-04,2,82,0
880,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-11,3,72,0
1197,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-18,4,77,0
1514,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-25,5,87,0
...,...,...,...,...,...,...,...,...
10793,2000,matchbox twenty,4:12,Bent,2000-12-23,35,33,316
11110,2000,matchbox twenty,4:12,Bent,2000-12-30,36,37,316
11427,2000,matchbox twenty,4:12,Bent,2001-01-06,37,38,316
11744,2000,matchbox twenty,4:12,Bent,2001-01-13,38,38,316


,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87
5,0,2000-04-01,94
6,0,2000-04-08,99
7,1,2000-09-02,91
8,1,2000-09-09,87
9,1,2000-09-16,92


## Multiple variables stored in one column

### Tubercolosis Example

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In [13]:
df_tb = pd.read_csv(path_to_data + "tb-raw.csv")
df_tb

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


In [14]:
df_tb_formatted = pd.melt(df_tb, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")

# Extract Sex, Age lower bound and Age upper bound group
tmp_df = df_tb_formatted["sex_and_age"].str.extract("(\D)(\d+)(\d{2})")    

tmp_df


# # Name columns
# tmp_df.columns = ["sex", "age_lower", "age_upper"]

# # Create `age`column based on `age_lower` and `age_upper`
# tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

# # Merge 
# df_tb_formatted = pd.concat([df_tb_formatted, tmp_df], axis=1)

# # Drop unnecessary columns and rows
# df_tb_formatted = df_tb_formatted.drop(['sex_and_age',"age_lower","age_upper"], axis=1)

# #df_tb_formatted = df_tb_formatted.dropna()

# df_tb_formatted = df_tb_formatted.sort_values(ascending=True,by=["country", "year", "sex", "age"])
# df_tb_formatted.head(15)


,0,1,2
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14
...,...,...,...
85,f,0,14
86,f,0,14
87,f,0,14
88,f,0,14


## 4. Variables are stored in both rows and columns

### Global Historical Climatology Network Dataset

In [15]:
df_weather = pd.read_csv(path_to_data + "weather-raw.csv")
df_weather

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_weather_formatted = pd.melt(df_weather, id_vars=["id", "year","month","element"], var_name="day_raw")
df_weather_formatted.head(10)

,id,year,month,element,day_raw,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
5,MX17004,2010,3,tmin,d1,NaN
6,MX17004,2010,4,tmax,d1,NaN
7,MX17004,2010,4,tmin,d1,NaN
8,MX17004,2010,5,tmax,d1,NaN
9,MX17004,2010,5,tmin,d1,NaN


In [17]:
# Extracting day
df_weather_formatted["day"] = df_weather_formatted["day_raw"].str.extract("d(\d+)", expand=False)  
df_weather_formatted["id"] = "MX17004" # Some ids had spaces e.g. 'MX17004 '
# df_weather_formatted.head(10)

# To numeric values - e.g.  year  month day
#                           2010      1   1
df_weather_formatted[["year","month","day"]] = df_weather_formatted[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))
# df_weather_formatted

# Creating a date from the different columns
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

df_weather_formatted["date"] = df_weather_formatted.apply(lambda row: create_date_from_year_month_day(row), axis=1)
#df_weather_formatted
df_weather_formatted = df_weather_formatted.drop(['year',"month","day", "day_raw"], axis=1)

df_weather_formatted = df_weather_formatted.dropna() # drops any row with even 1 NA 
#df_weather_formatted

# Unmelting column "element"
df_weather_formatted = df_weather_formatted.pivot_table(index=["id","date"], columns="element", values="value")
#df_weather_formatted.columns.name
df_weather_formatted

#df_weather_formatted.reset_index(drop=False, inplace=True)

# df_weather_formatted

element             tmax  tmin
id      date                  
MX17004 2010-02-02  27.3  14.4
        2010-02-03  24.1  14.4
        2010-03-05  32.1  14.2

## 5. One type in multiple tables

### Baby Names in Illinois

In [18]:
def extract_year(string):
    match = re.match(".+(\d{4})", string) 
    if match != None: 
        return match.group(1)
    
path = path_to_data
allFiles = glob.glob(path + "/201*-baby-names-illinois.csv")
print('allFiles', allFiles)

frame = pd.DataFrame()
df_list= []
for file_ in allFiles:
    df = pd.read_csv(file_)
    df.head(5)
    df["year"] = extract_year(file_)
    df_list.append(df)

df_list[0]
df_list[1]
    
df = pd.concat(df_list)
df = df.sort_values(by=["year", "rank"])
df

allFiles ['/Users/ortald/Desktop/AMLH-2023/tidy-data/data/2015-baby-names-illinois.csv', '/Users/ortald/Desktop/AMLH-2023/tidy-data/data/2014-baby-names-illinois.csv']


,rank,name,frequency,sex
0,1,Noah,863,Male
1,2,Liam,709,Male
2,3,Alexander,703,Male
3,4,Jacob,650,Male
4,5,William,618,Male


,rank,name,frequency,sex
0,1,Noah,837,Male
1,2,Alexander,747,Male
2,3,William,687,Male
3,4,Michael,680,Male
4,5,Liam,670,Male


,rank,name,frequency,sex,year
0,1,Noah,863,Male,2015
1,2,Liam,709,Male,2015
2,3,Alexander,703,Male,2015
3,4,Jacob,650,Male,2015
4,5,William,618,Male,2015
...,...,...,...,...,...
95,96,Giovanni,168,Male,2015
96,97,Hudson,167,Male,2015
97,98,Camden,165,Male,2015
98,99,Max,164,Male,2015


,rank,name,frequency,sex,year
0,1,Noah,837,Male,2014
1,2,Alexander,747,Male,2014
2,3,William,687,Male,2014
3,4,Michael,680,Male,2014
4,5,Liam,670,Male,2014
...,...,...,...,...,...
96,97,Max,170,Male,2014
97,98,Brody,168,Male,2014
98,99,Jaxson,168,Male,2014
99,100,George,166,Male,2014


,rank,name,frequency,sex,year
0,1,Noah,837,Male,2014
1,2,Alexander,747,Male,2014
2,3,William,687,Male,2014
3,4,Michael,680,Male,2014
4,5,Liam,670,Male,2014
...,...,...,...,...,...
95,96,Giovanni,168,Male,2015
96,97,Hudson,167,Male,2015
97,98,Camden,165,Male,2015
98,99,Max,164,Male,2015


## 6. Variables Are Stored In Both Rows And Columns 

### Raw Weather Dataset

In [19]:
df_weather = pd.read_csv("/Users/ortald/Desktop/AMLH-2023/tidy-data/data/weather-raw.csv")

df_weather_formatted = pd.melt(df_weather, id_vars=["id", "year", "month", "element"], var_name="day_raw")          # value_name – default is “value“

# Extracting day
df_weather_formatted["day"] = df_weather_formatted["day_raw"].str.extract("(\d+)")  
df_weather_formatted["id"] = "MX17004" # Some ids had spaces e.g., 'MX17004 ‘

# To numeric values - e.g.  year  month day
#                           2010      1   1
df_weather_formatted[["year", "month", "day"]] = df_weather_formatted[["year", "month", "day"]].apply(lambda x: pd.to_numeric(x, errors='ignore')) # If ‘ignore’, then invalid parsing will return the input   
df_weather_formatted
# Creating a date from the different columns
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=row["month"], day=row["day"])

# df_weather_formatted["date"] = df_weather_formatted.apply(lambda row: create_date_from_year_month_day(row), axis=1) # axis=1: apply function to each row 
df_weather_formatted["date"] = df_weather_formatted.apply(lambda row:create_date_from_year_month_day(row), axis=1) # axis=1: apply function to each row 


df_weather_formatted.drop(["year", "month", "day", "day_raw"], axis=1, inplace=True)
df_weather_formatted = df_weather_formatted.dropna() # drops rows with at least 1 NA
df_weather_formatted

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1
...,...,...,...,...,...,...,...
75,MX17004,2010,3,tmin,d8,NaN,8
76,MX17004,2010,4,tmax,d8,NaN,8
77,MX17004,2010,4,tmin,d8,NaN,8
78,MX17004,2010,5,tmax,d8,NaN,8


,id,element,value,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02
22,MX17004,tmax,24.1,2010-02-03
23,MX17004,tmin,14.4,2010-02-03
44,MX17004,tmax,32.1,2010-03-05
45,MX17004,tmin,14.2,2010-03-05


In [20]:
# Unmelting column "element” with pivot_table 
df_weather_formatted = df_weather_formatted.pivot_table(index=["id", "date"], columns="element", values="value")
df_weather_formatted

element             tmax  tmin
id      date                  
MX17004 2010-02-02  27.3  14.4
        2010-02-03  24.1  14.4
        2010-03-05  32.1  14.2

In [21]:
df_weather_formatted.reset_index(drop=False, inplace=True)
df_weather_formatted

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2
